<a href="https://colab.research.google.com/github/Charleo85/TFWorkshop/blob/master/tf_workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf;
print(tf.__version__)
import numpy as np
import random, math, datetime

# from Numpy to Tensor
- initialize tensor (n-dimentional array)

In [0]:
print(np.zeros((2, 1))) 
print(np.ones((1, 2), dtype=int))

In [0]:
print(tf.zeros([2, 1]))
print(tf.ones([1, 2], dtype=tf.int32))

> Notice there is no value but only shape and dtype in the tf tensor 
> untill you evaluate it in the computational graph

# Understand Session 

`tf.Session() ` encapsulates the environment in which Operation objects are executed, and Tensor objects are evaluated. 

In [0]:
print( tf.Session().run(tf.zeros([2, 1])) )
print( tf.Session().run(tf.ones([1, 2], dtype=tf.int32)) )

In [0]:
# Constant 1-D Tensor populated with value list.
tensor1d = tf.constant([1, 2, 5])
print(tensor1d)
# Constant 2-D tensor populated with scalar value -1.
tensor2d = tf.constant(-1.0, shape=[1, 3])
print(tensor2d)
# Create a tensor of shape [2, 3] consisting of random normal values, with mean -1 and standard deviation 4.
norm2d = tf.random_normal([2, 3], mean=-1, stddev=4)
print(norm2d)

with tf.Session() as sess:
  print(tensor1d.eval())
  print(tensor2d.eval())
  print(norm2d.eval())

- indexing and basic operation of tensor

In [0]:
a = np.zeros((2, 1))
b = np.ones((1, 2))
print(a[0,0], a[:,0], a[0,:])
np.reshape(a,(2,1))
print( a.shape)
print( np.sum(b, axis=1)) 

In [0]:
with tf.Session() as sess:
  a = tf.zeros([2, 1])
  b = tf.ones([1, 2])
  # indexing
  print( sess.run( (a[0,0], a[:,0], a[0,:]) ) )
  # reshape
  tf.reshape(a,(2,1))
  print(a.shape)
  # sum up along the 1st axis
  print(tf.reduce_sum(b, [1]).eval())

Note: Every tensor we define is by default saved in session

In [0]:
tf.Session().run( norm2d )

Use `tf.reset_default_graph()` to clean up tensor and the computational graph you have built so far

In [0]:
tf.reset_default_graph()

In [0]:
tf.Session().run( norm2d )
## RuntimeError is triggered as expected, since the tensor definition was cleared

# Build Flow

## Calculate ${(2*3)} ^ {2+3}$ with TensorFlow
### The 2-phase pipeline to code a tensorflow model
- Phase 1: assemble a graph, no computation happen at this phase!!!

In [0]:
x = tf.constant(2, name='2') # explicit naming the node help visualization in the graph
y = tf.constant(3, name='3')
op1 = tf.add(x, y, name='addition')
op2 = tf.multiply(x, y, name='muliply')
op3 = tf.pow(op2, op1, name='power')

- Phase 2: use a session to execute operations in the graph.

In [0]:
with tf.Session() as sess:
    print("2+3=", op1.eval()) 
    print("2*3=", op2.eval())
    print("%d^%d= %d"%(sess.run((op2, op1,op3)) ) )

## To visualize my computation graph in TensorBoard

In [0]:
with tf.Session() as sess:
    # save computational graph in directory used by tensorboard
    writer = tf.summary.FileWriter('./logs', sess.graph) 
writer.close() # close the writer when you’re done using it

If running in localhost, use command line
``` sh
$ tensorboard --logdir="./logs" --port 6006
# Then open your browser and go to: http://localhost:6006/
# you will see two graph one is the previous graph, the other is the current graph
```
Or load tensorboard in Colab:


In [0]:
# Load tensorboard extension in Colab
%load_ext tensorboard.notebook 

In [0]:
# let tensorboard listen on logs/ directory
%tensorboard --logdir logs/

## Calculate ${(x*3)} ^ {x+3}$ with TensorFlow
- use `tf.placeholder` to hold the place for input/output tensor 

In [0]:
g_1 = tf.Graph() # specify a graph scope to differetiate from previous graph definition
with g_1.as_default():
    x_placeholder = tf.placeholder(tf.int32, shape=(), name='x')
    y = tf.constant(3, name='3')
    op1 = tf.add(x_placeholder, y)
    op2 = tf.multiply(x_placeholder, y)
    op3 = tf.pow(op2, op1)

    with tf.Session() as sess:
        writer = tf.summary.FileWriter('./logs', sess.graph) 
        print(sess.run(op3, feed_dict={x_placeholder : 2}))
        print(sess.run(op3, feed_dict={x_placeholder : 10})) ## overflow happens

        print(sess.run(op3, feed_dict={x_placeholder : 3}))
    writer.close() # close the writer when you’re done using it

In [0]:
%tensorboard --logdir logs/

# Linear Regression with Tensorflow

In [0]:
### Simulate training Data in numpy
num_samples = 1000; # sample size
num_features = 5; # feture size
data_X = np.random.rand(num_samples, num_features)
weight = np.random.rand(num_features, 1)
noise = np.random.normal(0, .1, (num_samples, 1))
data_Y = np.dot(data_X, weight) + noise  #simulate a linear relation with gaussian noise
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(data_X, data_Y, test_size=0.1)

- use `tf.Variable` store the tensor whose value/state can be changed (trained)

In [0]:
# Graph Input Nodes
with tf.variable_scope("input/output", reuse=True):  # use variable scope to group graph node definition
    X = tf.placeholder(tf.float64, shape=(None, num_features), name="features")
    Y = tf.placeholder(tf.float64, shape=(None, 1), name="label")

In [0]:
# let model weight be Variable that is learnable
W = tf.Variable(np.random.rand(num_features, 1), name="weight")
b = tf.Variable(tf.constant(-1.0, shape=[1, 1], dtype=tf.float64), name="bias")

In [0]:
pred = tf.add(tf.matmul(X, W), b)    # Construct a linear model y = xW + b
cost = tf.reduce_mean(tf.pow(pred-Y, 2))    # Mean squared error

- `tf.train.optimizer` provide optimizer API to train a model https://www.tensorflow.org/api_docs/python/tf/train/Optimizer

In [0]:
# construct Gradient Descent optimizer
learning_rate = 0.1
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

- `tf.summary` provide a way to export condensed information about a model, which is then accessible in tools such as TensorBoard. https://www.tensorflow.org/versions/r1.1/api_guides/python/summary

In [0]:
with tf.variable_scope('linear_regression_summarizer', reuse=True):
    training_summary = tf.summary.scalar("training_cost", cost)
    evaluation_summary = tf.summary.scalar("evaulation_cost", cost)

In [0]:
num_epoch = 20
with tf.Session() as sess:
    # log info to visualize in tensorboard
    writer = tf.summary.FileWriter('./logs', sess.graph) 
    sess.run(tf.global_variables_initializer()) #initialize Variables
    for epoch in range(num_epoch):
        # compute on train subgraph
        _, trn_cost = sess.run([optimizer, training_summary], feed_dict={X: X_train, Y: Y_train})
        writer.add_summary(trn_cost, epoch) 
        # compute on evaluation subgraph
        eval_accuracy = sess.run(evaluation_summary, feed_dict={X: X_test, Y: Y_test})
        writer.add_summary(eval_accuracy, epoch) 

In [0]:
%tensorboard --logdir logs/

# Logistic Regression with tensorflow

MNIST: predict handwriting digits into labels of 0-9

### load MNIST dataset and split into train and test 

In [0]:
# Import MNIST data
import urllib
urllib.request.urlretrieve("https://s3.amazonaws.com/img-datasets/mnist.npz", "mnist.npz")
with np.load('mnist.npz') as f:
    x_train, y_train = f['x_train'], f['y_train']
    x_test, y_test = f['x_test'], f['y_test']

In [0]:
print(x_train.shape)
print(y_train.shape)

![feature](https://cdn-images-1.medium.com/max/800/1*Vo8PMHppg_lWxFAZHZzNGQ.png)

### Preprocess: Flatten 2d image into 1d

In [0]:
# convert x train from 2d image into 1d array of feature
num_features = x_train.shape[1]*x_train.shape[2]
x_train_flatten = x_train.reshape((x_train.shape[0], num_features)).astype(float)
# convert label to one hot embedding
nb_classes = 10
y_train_onehot = np.eye(nb_classes)[y_train.reshape(-1)]

In [0]:
print(x_train_flatten.shape)
print(y_train_onehot.shape)

In [0]:
# convert x train from 2d image into 1d array of feature
x_test_flatten = x_test.reshape((x_test.shape[0], x_test.shape[1]*x_test.shape[2])).astype(float)
# convert label to one hot embedding
nb_classes = 10
y_test_onehot = np.eye(nb_classes)[y_test.reshape(-1)]

In [0]:
print(x_test_flatten.shape)
print(y_test_onehot.shape)

### TODO: Contruct multi-class Logistic Regression model in Tensorflow

The probability that an input vector x is a member of a class $i$, a value of a stochastic variable $Y$, can be written as:

![likehood function](http://deeplearning.net/tutorial/_images/math/53ea9da92e0696d179c66a7db8ee113c9ffff68e.png)

In [0]:
# clean up previous graph
tf.reset_default_graph()

1. set placeholder for input feature and output label

In [0]:
# tf Graph Input
x = tf.placeholder(tf.float32, [None, 784]) # mnist data image of shape 28*28=784 for feature
y = tf.placeholder(tf.float32, [None, 10]) # 0-9 digits recognition => 10 classes for label

In [0]:
with tf.name_scope('input_reshape'):
  image_shaped_input = tf.reshape(x, [-1, 28, 28, 1])
  input_image_summary = tf.summary.image('input', image_shaped_input, 10)

2. set Variable to train in logistic regression model

In [0]:
# Set model weights
W = 
b = 

3. construct model 

`tf.nn.softmax()` https://www.tensorflow.org/api_docs/python/tf/nn/softmax

In [0]:
# Construct model
y_hat = 

Cross entropy loss
$$ l = -\sum_i y_i log \hat{y_i} $$

In [0]:
# Minimize error using cross entropy
cost = 

In [0]:
# Gradient Descent
trainstep =

The model’s prediction y_{pred} is the class whose probability is maximal, specifically:
![y_pred](http://deeplearning.net/tutorial/_images/math/26072fdfd3b18206ef38075d82dac9e394ecb6d7.png)

In [0]:
with tf.variable_scope('summarizer'):
    training_summary = tf.summary.scalar("training_cost", cost)
    correct_prediction = tf.equal(tf.argmax(y_hat, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32) )
    evaluation_summary = tf.summary.scalar("evaulation_accuracy", accuracy)

In [0]:
training_epochs = 10
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer()) #initialize Variables
    model_name = 'logistic_regression'
    now = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    writer = tf.summary.FileWriter('./logs/'+model_name+'_'+now, sess.graph) 
    input_summary = sess.run(input_image_summary, feed_dict={x: x_test_flatten, y: y_test_onehot})
    writer.add_summary(input_summary) 
    
    # Training cycle
    for epoch in range(training_epochs):
        _, trn_summmary = sess.run([trainstep, training_summary], feed_dict={x: x_train_flatten, y: y_train_onehot})
        writer.add_summary(trn_summmary, epoch) 
        
        eval_summary = sess.run(evaluation_summary, feed_dict={x: x_test_flatten, y: y_test_onehot})
        writer.add_summary(eval_summary, epoch) 
        
    writer.close()

In [0]:
%tensorboard --logdir logs/

# High-level API with keras
- api reference to https://www.tensorflow.org/api_docs/python/tf/keras
## multiclass logistic regression model with Keras

In [0]:
## if you want to remove tensorboard logs in previous run 
!rm -rf logs/

In [0]:
tf.keras.backend.clear_session()

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(), 
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

# try different loss, optimizers: sgd, adam
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# The patience parameter is the amount of epochs to check for improvement
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

tensor_board = tf.keras.callbacks.TensorBoard(log_dir='./logs/logistic_keras')

model.fit(x_train, y_train, epochs=100, validation_split=0.2, verbose=1, callbacks=[early_stop, tensor_board])

In [0]:
%tensorboard --logdir logs/

In [0]:
model.evaluate(x_test, y_test)


## Eager Execution and TensorFlow 2.0

Experiment feature in tensorflow 1.*
Now integrated in Tensorflow 2.0

In [0]:
tf.executing_eagerly() 